## How much money can a director lose before they disappear in the industry?

### How many box office bombs can an established director endure?

In [142]:
# 20200726 JLCY

import pandas as pd
import numpy as np
import math

In [143]:
xl = pd.ExcelFile('directors_data.xlsx')
len(xl.sheet_names)

50

In [144]:
xl.sheet_names[0]

'J.J. Abrams'

In [145]:
xl_file = pd.read_excel('directors_data.xlsx', sheet_name=xl.sheet_names[0])

# combine 1st 2 rows as column names
combine_col = []

for ind, col in enumerate(xl_file.columns): 
    
    if isinstance(xl_file.iloc[0, ind], str): 
        combine_col.append(col + ' ' + xl_file.iloc[0, ind])
    else: 
        combine_col.append(col)
        
combine_col = ['Opening Theaters' if i == 'Opening.1 Theaters' else i for i in combine_col]

xl_file.columns = combine_col

df = xl_file.iloc[1:-3] # drop 1st row and last 3 summary rows
df.reset_index(inplace=True)
df = df.iloc[:, 1:] # drop 1st duplicating column

# populate data for multiple roles in the same movie
nan_col = [col for col in df.columns if col not in ['Role']]

for i in df.index: 
    
    if type(df['Title'][i]) is float: 

        if math.isnan(df['Title'][i]) and isinstance(df['Role'][i], str): 
            
            for col in nan_col: 
                df[col][i] = df[col][i-1] # set data to the row above

In [146]:
df.head()

,Release Date,Title,Role,Distributor,Genre,Production Budget,Opening Weekend,Opening Theaters,Maximum Theaters,Theatrical Engagements,Domestic Box Office,International Box Office,Worldwide Box Office
0,2021-06-04 00:00:00,Micronauts,Producer,Paramount Pictures,Adventure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-20 00:00:00,Star Wars: The Rise of Skyw…,Director,Walt Disney,Adventure,275000000,177383864,4406,4406,29704,515202542,558267058,1073469600
2,2019-12-20 00:00:00,Star Wars: The Rise of Skyw…,Screenwriter,Walt Disney,Adventure,275000000,177383864,4406,4406,29704,515202542,558267058,1073469600
3,2019-12-20 00:00:00,Star Wars: The Rise of Skyw…,Producer,Walt Disney,Adventure,275000000,177383864,4406,4406,29704,515202542,558267058,1073469600
4,2018-11-09 00:00:00,Overlord,Producer,Paramount Pictures,Action,38000000,10202108,2859,2859,8038,21704844,17473345,39178189
